# 🎬 AI 기반 영상 컨텐츠 번역 자동화 프로젝트 1단계<br><br>


####1️⃣ 프로젝트 배경 및 목표
본 프로젝트는 드라마 제작사 및 배급사가 콘텐츠 수출 시 겪는 수작업 번역 및 재제작 비용 문제를 인공지능(AI)을 통해 해결하고자 기획되었습니다.  
- `취지`  
콘텐츠 수출을 위한 언어 현지화(번역) 과정에서 발생하는 높은 비용과 인력 소모를 AI 솔루션으로 대체하여 효율성을 극대화합니다.
- `비즈니스 잠재력`   
AI 기반 자동 번역 시스템 구축을 통해 한국콘텐츠진흥원(KOCCA)과 같은 유관 기관과의 비즈니스적 제휴 및 협력 모델을 창출할 수 있는 가능성을 확인하고자 실험을 진행했습니다.<br><br>  

####2️⃣ 테스트 환경 및 영상 데이터
AI 파이프라인의 성능을 검증하기 위해 실제 상업 드라마의 클립을 사용했습니다.
- 영상: 디즈니플러스 오리지널 드라마 "조각도시"
- 길이: 총 4분 2초 분량의 클립을 테스트 데이터로 사용했습니다.
- 특징: 한국어와 영어가 혼재된 환경을 가정하여 모델의 다국어 처리 능력을 중점적으로 테스트했습니다.<br><br>

####3️⃣ 음성 인식(ASR) 모델 선정 - Automatic Speech Recognition
복합적인 언어 환경을 처리하기 위해 OpenAI의 다국어 ASR 모델을 선택했습니다.  
🟡 모델: `openai/whisper-Large`
- 모델개요: OpenAI가 개발한 음성인식 / 번역모델 계열  
- 핵심구조: Transformer 기반의 인코더-디코더 아키텍쳐
- 학습데이터: 야 68만시간 분량의 멀티언어 음성데이터로 학습
- 선정이유: 드라마 영상에 한국어와 영어가 혼재되어 있으므로, 단일 언어전용모델이 아닌 다국어(Multilingual)지원이 필수적이었기 때문입니다.<br><br>

####4️⃣ 기계 번역(NMT) 모델 선정 - Neural Machine Translation
영어로 인식된 음성을 최종 결과물 언어인 한국어로 번역하기 위한 모델입니다.  
🟡 모델: `Helsinki-NLP/opus-mt-tc-big-en-ko`  
- 모델개요: 영어➔한국어 방향의 전문 기계 번역모델
- 핵심구조: Transformer-Big 아키텍쳐 기반
- 학습데이터: OPUS 라는 대규모 병렬 말뭉치(원문/번역 쌍) 기반으로, 특히 영어-한국어 병렬 데이터가 많음.
- 선정이유: 영어➔한국어 번역 지원이 명확하고, Hugging Face 의 pipeline('translation',...)함수와 즉시 연동가능, 무료 공개모델이며, 코랩환경에서 구동시 리소스 부담이 적어 효율적인 실험이 가능함.<br><br>  

####5️⃣ 최종 파이프라인 구조 및 모델 선택 이유
현재 구축된 자막 생성 파이프라인은 다음과 같은 단계로 구성되며, 모델은 이 구조에 최적화 되어 선택함.
- ASR(Whisper): 영상의 음성을 인식하여 한국어는 한국어로, 영어는 영어로 텍스트를 추출함.
- 언어분류: 추출된 텍스트 중 한국어와 영어를 분리합니다.
- 번역(OPUS-MT): 영어 텍스트만 선별하여 한국어로 번역을 수행합니다.
- 최종 통합: Whisper가 인식한 원래의 한국어 텍스트와 OPUS-MT가 번역한 영어의 한국어 텍스트를 합쳐 최종 자막(모두 한국어)을 완성<br><br>

####6️⃣ 최종 파이프라인 흐름
1. 영상에서 오디오 추출(mp4➔wav)  
2. Whisper 로 시간 정보와 함께 음성인식  
3. 인식된 텍스트 중 영어 대사만 골라 한국어로 번역  
4. 모든 한국어 텍스트와 시간 정보를 조합하여 SRT 자막파일 생성  
5. FFmpeg 를 이용해 영상에 자막을 최종 적용.<br><br>


#### ① 환경 세팅(라이브러리, GPU, 드라이브, 폰트)
##### - 패키지 설치 % 허깅페이스 로그인 & 기본 import

In [1]:
!pip install -U "huggingface_hub" "transformers" "accelerate" "sentencepiece" -q
"""
U는 Upgrade: 이미 있으면 버전 올리기
accelerate : gpu/cpu 효율처리
"""

from google.colab import userdata
from huggingface_hub import login

# Secrets에 저장된 HF_TOKEN 값을 가져와 login 함수에 전달
hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)

# 또는, 토큰이 환경 변수로 설정되면 login()만 호출해도 됩니다.
# import os
# os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
# login()

import torch
import textwrap  # 긴 문자열을 일정 길이로 잘라주는 유틸
import os
import shutil

from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
"""
AutoModelForSpeechSeq2Seq : 음성입력->시퀀스(텍스트) 출력 모델 자동로더
AutoProessor : 오디오 전처리 + 토크나이저를 한번에 묶어주는 객체
pipeline: 작업별로 모델+토크나이저를 간편하게 묶어서 쓰는 래퍼, asr파이프라인이나 번역파이프라인같이 쉽게 쓸수 있게 해주는 어떤 틀.
"""

print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU name:', torch.cuda.get_device_name(0))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 137.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 32.1 MB/s eta 0:00:00
CUDA available: True
GPU name: NVIDIA L4


##### - ffmpeg 설치 & 구글 드라이브 마운트

In [ ]:
!apt-get -q install ffmpeg

#ffmpeg : 영상/오디어 처리 툴
#apt-get : 리눅스 시스템 패키지 매니저(Ubuntu)

from google.colab import drive
drive.mount('/content/drive')

Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Mounted at /content/drive


##### - 배민 폰트 설치(한나체 프로)



In [ ]:
# 드라이브에 업로드한 폰트 경로 (네 드라이브 구조에 맞게 수정)
baemin_font_src = "/content/drive/MyDrive/fonts/BMHANNAPro.ttf"

if not os.path.exists(baemin_font_src):
    raise FileNotFoundError(f"배민 폰트를 찾을 수 없습니다: {baemin_font_src}")

# 리눅스 공용 폰트 폴더에 복사
!mkdir -p /usr/share/fonts/truetype/baemin  #mkdir -p : 폴더 없으면 만들고, 있으면 조용히 넘어감.
!cp "$baemin_font_src" /usr/share/fonts/truetype/baemin/  #cp : 파일복사

# 폰트 캐시 새로고침
!fc-cache -fv

# 설치된 폰트 이름 확인(ffmpeg force_style에서 사용)
!fc-list | grep -i hanna


/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/baemin: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/baemin: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directo

#### ② 드라마 영상에서 오디오 추출(mp4 ➔ wav)
##### - mp4 ➔ 16kHz mono wav

In [ ]:
video_path = "/content/drive/MyDrive/project_src/drama_themanipulated_4m2s.mp4"

if not os.path.exists(video_path):
    raise FileNotFoundError(f'영상 파일을 찾을수가 없어요: {video_path}')

# Whisper가 좋아하는 포맷, ffmpeg 로 오디오 추출(16kHz, mono wav)
audio_filename = 'audio_16k_mono.wav'

!ffmpeg -i '{video_path}' -vn -acodec pcm_s16le -ar 16000 -ac 1 "{audio_filename}"

# -i : input 입력파일
# -vn : video no. 즉, 비디오는 버리고 오디오만 추출
# -acodec pcm_s16le : 오디오 코덱 형식 지정(16-bit PCM)
# -ar 16000 : sample rate = 16kHz
# -ac 1 : mono(1채널)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

#### ③ Whisper로 음성 -> 텍스트(+타임스탬프)


In [ ]:
model_id = 'openai/whisper-large'
"""
small
medium
large 실험 후 최종 Large 사용
"""
device = 0 if torch.cuda.is_available() else 'cpu'
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32 #GPU 있을때 16비트 쓰면 보통 더 빠르고 메모리 절약

# 모델 로드(safetensors 체크포인트에서 가중치 읽어오기)
model = AutoModelForSpeechSeq2Seq.from_pretrained(  #허깅페이스에서 model_id에 해당하는 체크포인트 다운로드하고 로드
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
)

# 오디오 전처리 + 토크나이저를 묶어주는 Processor
processor = AutoProcessor.from_pretrained(model_id)


# ASR 파이프라인 정의
asr_pipe = pipeline(
    'automatic-speech-recognition', #ASR #미리 정의된 태스크 이름을 주면 내부에서 알아서 모델, 토크나이저, 전처리기를 묶어줌.
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor, #오디오를 모델 입력용 숫자 벡터로 바꿔줌.
    torch_dtype = torch_dtype,
    device=device,
    chunk_length_s=30, # 긴 오디오를 30초 단위 청크로 잘라서 처리
)

# 단어 단위 타임스탬프 얻기
asr_result = asr_pipe(
    audio_filename,
    return_timestamps="word"
)
"""
🟪 word-level timestamps
asr_result["chunks"]에,
{"text": "단어", "timestamp": (start, end)} 형태로 들어옴.
"""

print(asr_result.keys())
print('전체 텍스트 예시:', asr_result['text'][:200])
print('word-chunk 예시 10개:' , asr_result['chunks'][:10])


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See 

dict_keys(['text', 'chunks'])
전체 텍스트 예시:  피고인 측 대리인이 주장한 것과 같이 저희도 사건 당일 피고의 행적을 팔로우 했었습니다. 이 피고 박태중의 하루 일과를 5분이 단위로 찍은 GPS 동선입니다. 보시다시피 피고 측이 준비한 동선과 일치합니다. 그래서 저 역시 피고가 얼마나 성실한 청년인지에 대해서는 이의가 없습니다. 대한민국 헌법 제27조 제4항. 형사 피고인은 유죄의 판결이 확정될 때까지
word-chunk 예시 10개: [{'text': ' 피고인', 'timestamp': (0.0, 3.38)}, {'text': ' 측', 'timestamp': (3.38, 3.48)}, {'text': ' 대리인이', 'timestamp': (3.48, 3.9)}, {'text': ' 주장한', 'timestamp': (3.9, 4.4)}, {'text': ' 것과', 'timestamp': (4.4, 4.62)}, {'text': ' 같이', 'timestamp': (4.62, 4.9)}, {'text': ' 저희도', 'timestamp': (4.9, 5.54)}, {'text': ' 사건', 'timestamp': (5.54, 5.7)}, {'text': ' 당일', 'timestamp': (5.7, 6.12)}, {'text': ' 피고의', 'timestamp': (6.12, 6.96)}]


#### ④ "word"들을 드라마 자막 블럭으로 묶기


##### - word chunks ➔ 초벌 자막 블럭 리스트(raw_subs)

In [ ]:
"""
💬 group_words_to_subtitles는 단순히 글자 수/시간/침묵 기준으로만 묶어서,
문법이나 조사가 끊기지 않도록 “언어적”으로 이해하진 못하는게 한계다. 여기서 Whisper X 또는 사람손이 필요하다.
"""

def group_words_to_subtitles(
    word_chunks,
    max_chars: int = 60,      # 한 자막 블럭에 허용할 최대 글자 수
    max_duration: float = 7.0, # 한 자막 블럭의 최대 시간(초)
    max_gap: float = 2,     # 단어 사이 침묵이 2보다 크면 새 자막으로 끊기
    min_word_dur = 0.05, # 최소 단어 길이(초)
):
    """
    Whisper가 준 word-level chunk 들을 '자막 블럭'으로 묶어주는 함수.

    word_chunks: asr_result["chunks"]
      각 원소는 {"text": "...", "timestamp": (start, end)} 형태.

    반환: 자막 블럭 리스트
      [{"start": float, "end": float, "text": str}, ...]
    """
    subs = [] # 최종자막 블럭 리스트
    cur_words = [] # 현재 자막블럭에 들어갈 단어들
    cur_start = None # 이 블럭의 시작 시간
    last_end = None # 가장 마지막 단어의 끝 시간
    last_word = None

    for ch in word_chunks:
        w = ch.get("text", "")
        if w is None:
            continue
        w = w.strip()
        if not w:
            continue

        ts = ch.get("timestamp", None)
        if ts is None:
            continue
        s, e = ts
        if s is None or e is None:
            continue

        if e - s < min_word_dur:
            continue

        if last_word is not None:
            last_text, last_s, last_e = last_word
            if (
                w == last_text and
                abs(s - last_s) < 0.01 and
                abs(e - last_e) < 0.01
            ):
                continue

        # 첫 단어라면 새 자막 시작
        if cur_start is None:
            cur_start = s
            last_end = e
            cur_words = [w]
            continue

        # 현재 자막에 이 단어를 더했을 때를 가정
        candidate_text = " ".join(cur_words + [w])  # 리스트안의 문자열들을 " " 공백으로 이어붙이기
        duration = e - cur_start # duration: 현재 자막 블럭 길이(초)
        gap = s - last_end # 이전 단어 끝과 현재 단어 시작 사이 공백(침묵)

        need_new_sub = False

        # 1) 글자 수가 너무 길어지면 끊기
        if len(candidate_text) > max_chars:
            need_new_sub = True

        # 2) 시간도 너무 길어지면 끊기
        if duration > max_duration:
            need_new_sub = True

        # 3) 단어 사이에 침묵 구간이 길면 끊기
        if gap > max_gap:
            need_new_sub = True

        if need_new_sub:

            if len(cur_words) <= 2:
                cur_words.append(w)
                last_end = e
                continue

            # 그게 아니라면, 자막 하나 확정 / 지금까지 모인 단어들로 자막 하나 완성
            sub_text = " ".join(cur_words)
            subs.append({
                "start": cur_start,
                "end": last_end,
                "text": sub_text,
            })
            # 새 자막 시작
            cur_start = s
            cur_words = [w]
        else:
            # 기존 자막에 단어만 추가
            cur_words.append(w)

        last_end = e

    # 마지막 덩어리 처리
    if cur_words:
        sub_text = " ".join(cur_words)
        subs.append({
            "start": cur_start,
            "end": last_end,
            "text": sub_text,
        })

    return subs


word_chunks = asr_result["chunks"]
raw_subs = group_words_to_subtitles(
    word_chunks,
    max_chars=60,
    max_duration=7.0,
    max_gap=2,
    min_word_dur =0.05
)

print("초벌 자막 블럭 개수:", len(raw_subs))
print("예시 30개:")
for sub in raw_subs[:30]:
    print(sub)


초벌 자막 블럭 개수: 32
예시 30개:
{'start': 0.0, 'end': 6.96, 'text': '피고인 측 대리인이 주장한 것과 같이 저희도 사건 당일 피고의'}
{'start': 6.96, 'end': 13.26, 'text': '행적을 팔로우 했었습니다. 이 피고 박태중의 하루 일과를 5분이 단위로 찍은 GPS'}
{'start': 13.26, 'end': 20.2, 'text': '동선입니다. 보시다시피 피고 측이 준비한 동선과 일치합니다. 그래서 저 역시 피고가'}
{'start': 20.2, 'end': 26.86, 'text': '얼마나 성실한 청년인지에 대해서는 이의가 없습니다. 대한민국 헌법 제27조 제4항. 형사 피고인은'}
{'start': 26.86, 'end': 33.86, 'text': '유죄의 판결이 확정될 때까지는 무죄로 추정된다. 의심스러울 때는 피고인의 이익으로. 그래서'}
{'start': 33.86, 'end': 40.72, 'text': '저는 개인적으로 피고 박태중씨가 무죄였으면 좋겠습니다. 무죄라고 생각하는 제 마음에'}
{'start': 42.0, 'end': 46.6, 'text': '걸리는 지점 몇 가지만 피고가 해소시켜주시면 감사하겠습니다. 그래 주실 수 있나요?'}
{'start': 48.84, 'end': 55.38, 'text': '네. 자, 첫 번째 의문점입니다. 피고가'}
{'start': 55.38, 'end': 61.94, 'text': '피해자의 집에 배달을 갔다는 사실입니다. 물론 뭐'}
{'start': 61.94, 'end': 67.54, 'text': '무작위로 배달하다 보면 피해자의 집에 배달 갈 수 있습니다, 같은 행정구역에 살고 있으니까요. 그런데 제가'}
{'start': 67.54, 'end': 73.78, 'text': '궁금한 건 바로 배달 시간입니다. PHI 집에 오토바이가 도착하고 다시'}
{'start': 73.78, 'end': 80.44, 'text': '출발

#### ⑤ 영어만 번역 ➔ 한국어 통합 + 후처리(postprocess)
##### - 번역 파이프라인 + 후처리 함수 + processed_chunks 생성

In [ ]:
device = 0 if torch.cuda.is_available() else "cpu"

# 영어 ➔ 한국어 번역 파이프라인
translator = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-tc-big-en-ko",
    device=device,
)

# 번역 테스트
test = translator("I am testing English to Korean translation.")[0]["translation_text"]
print("테스트 번역:", test)


# 번역 필요 여부 판단
def contains_hangul(text: str) -> bool:  #문자열에 한글(가~힣)이 하나라도 포함되어 있으면 'True'
    for ch in text:
        if "\uac00" <= ch <= "\ud7a3":
            return True
    return False

def needs_translation(text: str) -> bool:

    if contains_hangul(text):   # 한글이 포함되어 있으면 이미 한국어가 섞여 있다고 보고 번역하지 않음.
        return False

    if any(c.isalpha() for c in text):   # 한글은 없는데 알파벳만 있으면 영어라고 보고 번역
        return True
    return False                        # 그 외(숫자, 기호): 번역 안함


# ASR이 이상하게 인식한 표현 후처리
def postprocess_ko_text(text: str) -> str:
    fixes = {
        #✅small
        # "빨라고": "팔로우",
        # "혼법" : "헌법",
        # "의미인" : "의문",
        # "갖다는" : "갔다는",
        # "페이지아이" : "피해자의",
        # "행정구에게" : "행정구역에",
        # "서유" : "소요",
        # "비되면" : "비대면",
        # "감시" : "감식",
        # "이입" : "이의있",
        # "지키지" : "찍히지",
        # "사각지 들으면" : "사각지대로만",
        # "피고치게 이해" : "피고측의 이의에",
        # "피지" : "피의자",
        # "알려가지고" : "하셔가지고",
        #✅medium
        # "의미점" : "의문점",
        # "PHI" : "피해자",
        # "철문희" : "젊은이",
        # "하셔서요" : "하셔가지고요",
        # "감시" : "감식",
        # "예 있습니다" : "이의 있습니다"
        # "지키지" : "찍히지",
        # "이해" : "이의에",
        # "하려가지고" : "하셔가지고",
        #✅large
        "PHI" : "피해자의",
        "피고 측에 의해" : "피고측의 이의에",
    }
    for wrong, correct in fixes.items():
        text = text.replace(wrong, correct)
    return text


def merge_subtitles_by_sentence(chunks,
                               max_merged_chars: int = 60,
                               max_time_gap: float = 1.2):
    """
    processed_chunks를 받아서,
    문장 중간에 잘린 것 같은 자막들을 다시 합쳐주는 후처리.

    - 이전 자막이 문장 끝(., ?, !, '다.', '요.') 등으로 끝나지 않고
    - 두 자막 사이의 시간 간격이 너무 크지 않으며
    - 둘을 합쳤을 때 글자 수가 max_merged_chars 이하이면

    → 하나의 자막 덩어리로 병합한다.
    """
    if not chunks:
        return []

    merged = [chunks[0].copy()]

    for cur in chunks[1:]:
        prev = merged[-1]
        prev_text = prev["text"].rstrip()
        cur_text = cur["text"].strip()

        sentence_endings = ("다.", "요.", "까?", "냐?", "니다.", ".", "?", "!")
        if prev_text.endswith(sentence_endings):
            merged.append(cur.copy())
            continue

        gap = cur["start"] - prev["end"]
        if gap > max_time_gap:
            merged.append(cur.copy())
            continue

        candidate = prev_text + " " + cur_text
        if len(candidate) > max_merged_chars:
            merged.append(cur.copy())
            continue

        # 위 조건 다 통과하면 병합
        prev["end"] = cur["end"]
        prev["text"] = candidate

    return merged




def refine_timing(chunks,
                  max_display: float = 5.0,
                  min_gap: float = 0.15,
                  first_min_start: float = 2.0,
                  tail_pad: float = 1.5):
    """
    자막 타이밍 로컬 정리용 함수.

    - 첫 자막은 최소 first_min_start초 이후에 나오게 보정
      (영상 켜자마자 자막 떠 있는 문제 방지)

    - 각 자막 블럭은 최대 max_display초 이상 화면에 머무르지 않게 자름
      (회상이 길어져도 이전 자막이 너무 오래 붙어 있지 않게)

    - 이전 자막과 너무 겹치지 않도록 min_gap만큼 간격 확보
      (겹침/밀착으로 인한 어색함 완화)
    """

    if not chunks:
        return []

    refined = []

    for i, ch in enumerate(chunks):
        start = float(ch["start"])
        end = float(ch["end"])
        text = ch["text"]

        # 1) 첫 자막: 너무 0초에 붙어있으면 살짝 뒤로
        if i == 0 and start < first_min_start:
            shift = first_min_start - start
            start += shift
            end += shift

        # 2) 한 자막이 너무 오래 있으면 잘라주기
        dur = end - start
        if dur > max_display:
            end = start + max_display
            dur = max_display

        # 3) 이전 자막과의 간격 정리
        if refined:
            prev = refined[-1]
            # 이전 자막보다 너무 앞에서 시작하면, 조금 뒤로 밀기
            if start < prev["end"] + min_gap:
                start = prev["end"] + min_gap

            # 만약 이렇게 밀다가 end <= start가 되어버리면 최소 길이 확보
            if end <= start:
                end = start + 0.3  # 최소 0.3초 정도는 보여주기

        # 4) 자막이 너무 빨리 꺼지지 않게, tail_pad만큼 늘려주기
        end = end + tail_pad

        refined.append({
            "start": start,
            "end": end,
            "text": text,
        })

    return refined


# raw_sub에 번역 + 후처리 적용
processed_chunks = []

for sub in raw_subs:
    text = sub["text"].strip()
    if not text:
        continue

    # 영어만 번역
    if needs_translation(text):
        tr = translator(text)[0]["translation_text"]
        ko_text = tr.strip()
    else:
        ko_text = text

    # 후처리로 ASR 오타 교정
    ko_text = postprocess_ko_text(ko_text)

    processed_chunks.append({
        "start": sub["start"],
        "end": sub["end"],
        "text": ko_text,
    })



# 문장 단위 병합
processed_chunks = merge_subtitles_by_sentence(
    processed_chunks,
    max_merged_chars=60,
    max_time_gap=1.2,
)

# 타이밍 정리
processed_chunks = refine_timing(
    processed_chunks,
    max_display=5.0,   # 자막 한 블럭이 화면에 최대 5초 남게
    min_gap=0.15,      # 서로 0.15초 정도는 띄워주기
    first_min_start=2.0,  # 첫 자막은 최소 2초 이후에 나오게
)



print("처리된 자막 블럭 수:", len(processed_chunks))
print("예시 10개:")
for ch in processed_chunks[:10]:
    print(ch)



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/418M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/815k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


테스트 번역: 미국 심리학 1940 heart330.
처리된 자막 블럭 수: 28
예시 10개:
{'start': 2.0, 'end': 8.5, 'text': '피고인 측 대리인이 주장한 것과 같이 저희도 사건 당일 피고의'}
{'start': 8.65, 'end': 13.46, 'text': '행적을 팔로우 했었습니다. 이 피고 박태중의 하루 일과를 5분이 단위로 찍은 GPS'}
{'start': 13.610000000000001, 'end': 19.759999999999998, 'text': '동선입니다. 보시다시피 피고 측이 준비한 동선과 일치합니다. 그래서 저 역시 피고가'}
{'start': 20.2, 'end': 26.7, 'text': '얼마나 성실한 청년인지에 대해서는 이의가 없습니다. 대한민국 헌법 제27조 제4항. 형사 피고인은'}
{'start': 26.86, 'end': 33.36, 'text': '유죄의 판결이 확정될 때까지는 무죄로 추정된다. 의심스러울 때는 피고인의 이익으로. 그래서'}
{'start': 33.86, 'end': 40.36, 'text': '저는 개인적으로 피고 박태중씨가 무죄였으면 좋겠습니다. 무죄라고 생각하는 제 마음에'}
{'start': 42.0, 'end': 48.1, 'text': '걸리는 지점 몇 가지만 피고가 해소시켜주시면 감사하겠습니다. 그래 주실 수 있나요?'}
{'start': 48.84, 'end': 55.34, 'text': '네. 자, 첫 번째 의문점입니다. 피고가 피해자의 집에 배달을 갔다는 사실입니다. 물론 뭐'}
{'start': 61.94, 'end': 68.44, 'text': '무작위로 배달하다 보면 피해자의 집에 배달 갈 수 있습니다, 같은 행정구역에 살고 있으니까요. 그런데 제가'}
{'start': 68.59, 'end': 74.04, 'text': '궁금한 건 바로 배달 시간입니다. 피해자의 집에 오토바이가 도착하고 다시'}


#### ⑥ SRT 생성 + 드라이브에 저장
##### - SRT 시간 문자열, 긴 문장 줄바꿈, 파일저장

In [ ]:
# 초(float) ➔ SRT 시간 문자열 'HH:MM:SS,mmm'
def sec_to_srt_time(sec: float) -> str:
    if sec is None:
        sec = 0.0

    millis = int(round(sec * 1000))

    hours = millis // (3600 * 1000)  # //는 정수 나눗셈 -> 몫만 얻고 나머지는 버리기
    millis = millis % (3600 * 1000)  # %는 나머지만

    minutes = millis // (60 * 1000)
    millis = millis % (60 * 1000)

    seconds = millis // 1000
    millis = millis % 1000

    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{millis:03d}"  # 'HH:MM:SS,mmm' 형식으로 반환


# 긴 자막 문장을 영화 자막처럼 1~2줄 정도로 잘라주는 함수
def wrap_subtitle_text(text: str,
                       width: int = 25,  #width: 대략 한 줄당 허용 글자 수
                       max_lines: int = 2) -> list[str]:  #max_lines: 허용 줄 수(보통 영화 자막은 2줄 이하)

    cleaned = " ".join(text.split()) #공백정리 (여러공백->한 칸)
    lines = textwrap.wrap(cleaned, width=width) # 자막 한줄당 대략 25자 정도로 적당히 잘라서 리스트로 반환

    if len(lines) <= max_lines:    #줄 수가 max_lines 이하이면 그대로 반환
        return lines

    #줄 수가 너무 많으면, 앞의 (max_lines-1)줄 + 나머지를 마지막 줄에 합치기
    head = lines[:max_lines-1]
    tail = " ".join(lines[max_lines-1:]) # " ".join(...) : 다시 한칸씩만 붙이기 -> 중복 공백 정리
    return head + [tail]


# processed_chunks 형태의 리스트를 받아서 표준 SRT 형식의 파일로 저장
def save_srt(chunks, srt_filename="drama_ko_themanupulated.srt"):
    lines = []
    idx = 1

    for ch in chunks:
        text = ch['text'].strip()
        if not text:
            continue

        start = sec_to_srt_time(ch['start'])
        end = sec_to_srt_time(ch['end'])

        # ✅ 여기서 긴 문장을 1~2줄로 자르기
        wrapped_lines = wrap_subtitle_text(text, width=22, max_lines=2)

        lines.append(str(idx))
        lines.append(f"{start} --> {end}")
        # 한 자막 블럭 안에서도 여러 줄 넣어도 됨
        for wline in wrapped_lines:
            lines.append(wline)
        lines.append('')

        idx += 1

    srt_content = '\n'.join(lines)

    with open(srt_filename, 'w', encoding='utf-8') as f:
        f.write(srt_content)

    print(f'SRT 파일 저장 완료: {srt_filename}')


# 실제 SRT 파일 저장
srt_local = "drama_ko_themanupulated.srt"
save_srt(processed_chunks, srt_local)


# 구글드라이브에 백업
dst_path = '/content/drive/MyDrive/project_src/drama_ko_themanupulated.srt'
shutil.copy('drama_ko_themanupulated.srt', dst_path)

print('드라이브 저장완료:', dst_path)


SRT 파일 저장 완료: drama_ko_themanupulated.srt
드라이브 저장완료: /content/drive/MyDrive/project_src/drama_ko_themanupulated.srt


#### ⑦ 자막을 영상에 입히기 & 스타일 지정
##### - ffmpeg로 SRT를 영상에 구워 넣기

In [ ]:
# 드라이브에 있는 원본 영상 경로 (앞에서 썼던거 그대로)
video_path = '/content/drive/MyDrive/project_src/drama_themanipulated_4m2s.mp4'

if not os.path.exists(video_path):
    raise FileNotFoundError(f"영상 파일을 찾을 수 없습니다: {video_path}")

# 방금 만든 SRT
srt_path = "drama_ko_themanupulated.srt"
if not os.path.exists(srt_path):
    raise FileNotFoundError(f"SRT 파일을 찾을 수 없습니다: {srt_path}")


# 자막이 입혀진 결과 영상을 드라이브에 저장할 경로
output_video_path = "/content/drive/MyDrive/project_src/drama_themanipulated_4m2s_ko_sub.mp4"

print("원본 영상:", video_path)
print("사용할 SRT:", srt_path)
print("출력 영상:", output_video_path)

# 폰트 이름은 fc-list에서 확인한 값 사용
style = (
    "FontName=BM Hanna Air,"  # 배민 한나체 Air
    "FontSize=25,"            # 글자 크기
    "PrimaryColour=&H00FFFFFF,"   # 흰색 (BGR + 알파)
    "OutlineColour=&H00000000,"   # 검정 외곽선
    "Outline=2,"               # 외곽선 두께
    "BorderStyle=1,"           # 일반 테두리
    "Shadow=0,"                # 그림자 없음
    "Alignment=2,"             # 2 = 아래 가운데
    "MarginV=40"               # 화면 아래에서 위로 40px 여백
)
style = "".join(style)


# ffmpeg로 하드서브(영상에 자막 굽기)
!ffmpeg -y -i "$video_path" \
  -vf "subtitles=$srt_path:force_style='$style'" \
  -c:a copy "$output_video_path"


print("저장완료!")


원본 영상: /content/drive/MyDrive/project_src/drama_themanipulated_4m2s.mp4
사용할 SRT: drama_ko_themanupulated.srt
출력 영상: /content/drive/MyDrive/project_src/drama_themanipulated_4m2s_ko_sub.mp4
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshin